# Group5_Resume_Tracking_System


#### Anand Pratap Singh (12020084)
#### Francis David Vuppuluri (12020028)
#### Lavanya SN (12020075)
#### Puneet Srivastava (12020026)
#### Sangeeta Thakur (12020088)

In [78]:
import pandas as pd
from fuzzywuzzy import fuzz
from utils.simplifiedSkillList import replaceList

pd.set_option('max_colwidth', 400)

# read train data that was subjected to simplification of skillsets
train = pd.read_csv('./data/SimplifiedTrainData.tsv', sep='\t')

# read test data that was attached with predicted salaries
test = pd.read_csv('./data/PredictSalaryEnriched.tsv', sep='\t')

In [79]:
# creating a primary key for job-opening data
train['ID'] = train.reset_index()['index'].map(lambda x: 'J'+ str(x+1)) 

In [80]:
# converting the maximum offered salary to lakhs, basically to scale it down.
train['Salary'] = train['Salary']/100000

# removing the word "management" as it skews the results
def getSkillset(s):
    cleanedList = []
    for i in s.split(', '):
        cleanedList.append(i.replace('management','').strip())
    
    return ", ".join(cleanedList)

# some itsy bit cleaning of skillsets in candidate data
test['Skillset'] = test['Skillset'].map(lambda x: getSkillset(x))

### Here we are trying to create a mapping of all the candidates with the top 5 most relevant jobs for them. 

#### Firstly the predicted salaries are compared against the salaries in the given range and checked if they life in the range of maximum 1.2 times the max offered salary, and minimum of 0.7 times the max offered salary. From that small subset of matched data, we do a fuzzy match of the skill text and find the top 5 jobs for each candidate.




#### Following is the code for creation of a dictionary "finalCandidateJobMapper" that keeps a mapping of CandidateID and JobID.

In [81]:
trainSalaryTracker = dict()

for index, row in train.iterrows():
    trainSalaryTracker[row['ID']] = (row['Salary'], row['Simplified Skillset'])

In [86]:
candidateJobMapper = dict()

finalCandidateJobMapper = dict() # contains the mapping of Candidate and Job with a matching score

for index, row in test.iterrows():
    jobs = set()
    for key, value in trainSalaryTracker.items():

        if (value[0]*1.2) > row['Salary LR'] and (value[0]*0.7) < row['Salary LR']:
            jobs.add(key)
            candidateJobMapper[row['Index']] = jobs

In [87]:
for index, row in test.iterrows():
    jobs = set()      
    for jobKey in candidateJobMapper[row['Index']]:
        if fuzz.token_sort_ratio(trainSalaryTracker[jobKey][1], row['Skillset']) > 10:
            jobs.add((jobKey, fuzz.token_sort_ratio(trainSalaryTracker[jobKey][1], row['Skillset']))) 

            finalCandidateJobMapper[row['Index']] = jobs
#     print(row['Index'])

C1
C2
C3
C4
C5
C6
C7
C8
C9
C10
C11
C12
C13
C15
C16
C17
C18
C19
C20
C21
C22
C23
C24
C25
C26
C27
C28
C29
C30
C31
C32
C33
C35
C36
C38
C39
C40
C41
C42
C43
C44
C45
C46
C47
C48
C49
C50
C51
C52
C53
C54
C55
C56
C57
C58
C59
C60
C62
C64
C65
C66
C67
C68
C69
C70
C71
C72
C73
C74
C75
C76
C77
C79
C80
C82
C83
C84
C85
C86
C88
C89
C90
C91
C92


In [88]:
# we finally trim the candidate-job mapping to the top 5 results based on the salary and skill match criteria

candidateToJob = dict()

for key, value in finalCandidateJobMapper.items():
    candidateToJob[key] = sorted(value, key=lambda x:x[1], reverse=True)[0:5] if len(value) > 5 else sorted(value, key=lambda x:x[1], reverse=True)

In [89]:
test[test['Index'] == 'C9']

,Index,Skillset,Experience,Zone,Salary LR
8,C9,"algorithm, machine learning, data mining, computer science, structure, distributed systems, database, sql, oop, python, java, xml, mapreduce, orm, natural language processing",22,{'south'},55.544787


In [7]:
candidateJobMapper

{'C1': {('J111', 16),
  ('J117', 22),
  ('J120', 24),
  ('J126', 20),
  ('J144', 17),
  ('J145', 20),
  ('J147', 28),
  ('J15', 19),
  ('J150', 33),
  ('J151', 16),
  ('J160', 19),
  ('J16397', 23),
  ('J16404', 16),
  ('J165', 29),
  ('J212', 26),
  ('J232', 13),
  ('J234', 25),
  ('J251', 21),
  ('J253', 23),
  ('J256', 15),
  ('J257', 21),
  ('J277', 22),
  ('J278', 25),
  ('J3', 21),
  ('J30', 24),
  ('J58', 20),
  ('J66', 32),
  ('J67', 34),
  ('J70', 18),
  ('J71', 21),
  ('J72', 11),
  ('J74', 17),
  ('J76', 18),
  ('J79', 13),
  ('J82', 17),
  ('J91', 22),
  ('J92', 34),
  ('J97', 12)},
 'C2': {('J1', 24),
  ('J101', 20),
  ('J102', 30),
  ('J104', 20),
  ('J105', 28),
  ('J112', 19),
  ('J114', 19),
  ('J116', 19),
  ('J124', 33),
  ('J129', 18),
  ('J130', 24),
  ('J132', 35),
  ('J141', 20),
  ('J142', 20),
  ('J152', 32),
  ('J157', 29),
  ('J16', 25),
  ('J16375', 25),
  ('J16383', 58),
  ('J16385', 21),
  ('J16387', 15),
  ('J16390', 26),
  ('J16392', 17),
  ('J16398', 30

### An example of the result generated by our model

#### First cell shows the candidate data
#### Second cell shows the top job match for the given candidate

In [90]:
candidateToJob['C33']

[('J23390', 65), ('J25123', 64), ('J25118', 64), ('J17299', 64), ('J77', 58)]

In [91]:
test[test['Index'] == 'C33']

,Index,Skillset,Experience,Zone,Salary LR
31,C33,"angular, asp, c, web serv, sql, net, asp, linq, java, jquery, ssrs, ssis",9,{'south'},18.774067


In [92]:
train[train['ID'] == 'J23390']

,Title,Company,Experience,Salary,Skillset,Zone,Simplified Skillset,ID
23389,Opening with MSys For MEAN Stack developer - Immediate Joiners,MSys Tech India Pvt. Ltd.,5.5,20.0,"['AngularJS', 'SQL', 'Hapi', 'Node', 'Mongo DB', 'Web services', 'TDD', 'Javascript']","{'south', 'east', 'west', 'north'}","angular, sql, api, node, mongo, web serv, tdd, java",J23390
